# Chains

Although a whole process may be asynchronous, sometimes the steps inside that process depend on the completion of the previous ones:

- Taking the Order: The customer's order must be recorded before anything else can proceed.
- Preparing the Base Drink: The drink's base must be ready before add-ons are applied.
- Adding Add-ons: Each add-on is applied sequentially to ensure the order is correct.
- Serving the Drink: Only once everything is prepared can the drink be served.

By using promise chaining, each step waits for the previous step to complete before proceeding, ensuring the process occurs in the required order while still being asynchronous. This approach simplifies the code and maintains clarity.

First let's define what an `Order` really is:

In [1]:
type Order = {
  id: number;
  customer: string;
  baseDrink: "Coffee" | "Fruit Juice" | "Smoothie";
  addOns: ("Ice" | "Syrup" | "Blending" | "Milk" | "Whipped Cream")[];
};


Now let's define all the individual steps involved in processing processing an order:

In [2]:
function takeOrder(order: Order): Promise<Order> {
  console.log(`Taking order from ${order.customer}: ${order.baseDrink} with ${order.addOns.join(", ")}`);
  return new Promise((resolve) =>
    setTimeout(() => resolve(order), 1500)
  );
}

function prepareBaseDrink(order: Order): Promise<Order> {
  console.log(`Preparing base drink (${order.baseDrink}) for ${order.customer}...`);
  let time = 1000
  switch (order.baseDrink) {
    case "Fruit Juice":
      time = 2000;
      break;
    case "Smoothie":
      time = 3000;
      break;
  }
  return new Promise((resolve) =>
    setTimeout(() => resolve(order), time)
  );
}

function handleAddOns(order: Order): Promise<Order> {
  return order.addOns.reduce((promise, addOn) => {
    return promise.then(() => {
      console.log(`Adding ${addOn} to ${order.customer}'s ${order.baseDrink}...`);
      return new Promise((resolve) => setTimeout(resolve, 500));
    });
  }, Promise.resolve()).then(() => order);
}

function serveDrink(order: Order): Promise<void> {
  console.log(`Serving ${order.baseDrink} with ${order.addOns.join(", ")} to ${order.customer}.`);
  return new Promise((resolve) =>
    setTimeout(() => resolve(), 800));
}


Finally, let's handle the proper order of these steps by chaining them together:

In [3]:
function processOrder(order: Order): Promise<void> {
  return takeOrder(order)
    .then(prepareBaseDrink)
    .then(handleAddOns)
    .then(serveDrink)
    .then(() => {
      console.log(`Order ${order.id} for ${order.customer} is complete.`);
    })
    .catch((error) => {
      console.error(`Failed to process order ${order.id}:`, error);
    });
}


Now let's see how that works out in practice:

In [4]:
const orders: Order[] = [
  { id: 1, customer: "Alice", baseDrink: "Smoothie", addOns: ["Blending", "Whipped Cream"] },
  { id: 2, customer: "Bob", baseDrink: "Coffee", addOns: ["Milk", "Syrup"] },
  { id: 3, customer: "Charlie", baseDrink: "Fruit Juice", addOns: ["Ice"] },
];

Promise.all(orders.map(processOrder))
  .then(() => {
    console.log("All orders are completed.");
  });

Taking order from Alice: Smoothie with Blending, Whipped Cream
Taking order from Bob: Coffee with Milk, Syrup
Taking order from Charlie: Fruit Juice with Ice


Promise { <pending> }

Preparing base drink (Smoothie) for Alice...
Preparing base drink (Coffee) for Bob...
Preparing base drink (Fruit Juice) for Charlie...
Adding Milk to Bob's Coffee...
Adding Syrup to Bob's Coffee...
Adding Ice to Charlie's Fruit Juice...
Serving Coffee with Milk, Syrup to Bob.
Serving Fruit Juice with Ice to Charlie.
Order 2 for Bob is complete.
Adding Blending to Alice's Smoothie...
Order 3 for Charlie is complete.
Adding Whipped Cream to Alice's Smoothie...
Serving Smoothie with Blending, Whipped Cream to Alice.
Order 1 for Alice is complete.
All orders are completed.
